In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, sys
import tensorflow as tf

2023-06-18 01:52:16.178784: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-18 01:52:16.351358: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-18 01:52:16.352344: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-18 01:52:17.317846: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
import matplotlib.pyplot as plt
from tensorflow import keras

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from tqdm import tqdm
import PIL
from PIL import Image, ImageOps
import cv2
from sklearn.utils import class_weight, shuffle
from keras.losses import binary_crossentropy
from pathlib import Path

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from keras.applications.imagenet_utils import preprocess_input

import warnings
warnings.filterwarnings("ignore")
IMG_SIZE_LOAD=64
NUM_CLASSES = 5
SEED = 77
TRAIN_NUM = 1000 # use 1000 when you just want to explore new idea, use -1 for full train

In [4]:
train_dir = '/media/amanpreet/Windows-SSD/Users/siaya/Downloads/resized_train/resized_train/'

In [5]:
p_resize = '/media/amanpreet/Windows-SSD/Users/siaya/Downloads/trainLabels.csv'
d_resize = pd.read_csv(p_resize)
d_resize.head(5)

,image,level
0,10_left,0
1,10_right,0
2,13_left,0
3,13_right,0
4,15_left,1


In [6]:
def turn2DF(path):
    dataframe_train = pd.read_csv(path)

    for i in range(len(dataframe_train)):
        dataframe_train=dataframe_train.replace(dataframe_train.loc[i,'image'], train_dir + str(dataframe_train.loc[i,'image']) + '.jpeg')

    dataframe_train.to_csv('mydata.csv', index=False) # index=False để không lưu index của DataFrame vào file CSV

In [7]:
def crop_image_from_gray(img,tol=7):
    if img.ndim == 2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol

        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

def load_ben_color(path, sigmaX=10):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (IMG_SIZE_LOAD, IMG_SIZE_LOAD))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)

    return image

In [10]:
from keras.applications.imagenet_utils import preprocess_input
def arr_img():
    for i in range(len(d_resize)):
        xdata = np.zeros((d_resize.shape[0], IMG_SIZE_LOAD, IMG_SIZE_LOAD, 3))
        path = train_dir  + str(d_resize.loc[i,'image']) + '.jpeg'
        image = cv2.imread(path)
        image = cv2.resize(image, (IMG_SIZE_LOAD, IMG_SIZE_LOAD))
        x = tf.keras.preprocessing.image.img_to_array(image)
        xdata[i] = x

    xdata = xdata / 255.0
    return xdata

In [ ]:
x_train = arr_img()
x_train.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

lb = LabelEncoder()
y = lb.fit_transform(d_resize['level'])
y = to_categorical(y, num_classes = NUM_CLASSES)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D

model = Sequential()

model.add(Conv2D(filters = 30, kernel_size = (5, 5), input_shape = (64, 64, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters = 15, kernel_size = (3, 3), activation = 'relu'))
model.add(Conv2D(filters = 15, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))

model.add(Dense(5, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['categorical_accuracy'])